# SmartFigure ↔ Plotly Viewport Sync — Test Notebook

This notebook verifies the **two-way viewport synchronization** feature between Plotly and `SmartFigure`, including:

- Live vs Home (“default”) viewport semantics
- Plotly relayout → SmartFigure range updates (without changing “home”)
- Programmatic range updates → update both live and “home”
- “Reset axes” behavior snapping back to “home” (and removing autoscale)
- Debounced resampling/recompute trigger (0.5s idle in production, flushable for tests)

> **Note:** If imports fail, adjust the candidate module list in the first code cell to match your local package layout.


## Stages covered

1. **Viewport state model:** `x_range/y_range` are *live* and `x_range_default/y_range_default` are *home*.
2. **Plotly integration:** state is pushed to Plotly without clobbering UI state (`uirevision` strategy).
3. **Relayout listener:** Plotly pan/zoom updates live ranges; “reset axes” restores home.
4. **Debounced recompute:** UI-driven viewport changes request recompute after idle; tests use a `flush()` hook.


## Environment and imports

In [ ]:
from __future__ import annotations

import importlib
import inspect
import sys
from typing import Any, Optional

print("Python:", sys.version)

# Adjust this list if your SmartFigure lives elsewhere.
CANDIDATE_MODULES = [
    "gu_toolkit.plugins.SmartFigure.SmartFigure",
    "gu_toolkit.plugins.SmartFigure",
    "gu_toolkit.SmartFigure",
    "SmartFigure",
]

SmartFigure = None
last_err: Optional[BaseException] = None

for mod_name in CANDIDATE_MODULES:
    try:
        mod = importlib.import_module(mod_name)
    except Exception as e:
        last_err = e
        continue
    if hasattr(mod, "SmartFigure"):
        SmartFigure = getattr(mod, "SmartFigure")
        print("Imported SmartFigure from:", mod_name)
        break

if SmartFigure is None:
    raise ImportError(
        "Could not import SmartFigure. Tried:\n"
        + "\n".join(f"  - {m}" for m in CANDIDATE_MODULES)
        + "\n\nLast error: "
        + repr(last_err)
    )

print("SmartFigure signature:", inspect.signature(SmartFigure))
print("OK ✅")


## Helpers

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Iterable, Tuple

Range = Tuple[float, float]

def assert_range_eq(a: Range, b: Range, *, tol: float = 0.0) -> None:
    assert len(a) == 2 and len(b) == 2
    if tol == 0.0:
        assert a[0] == b[0] and a[1] == b[1], f"Ranges differ: {a} vs {b}"
    else:
        assert abs(a[0] - b[0]) <= tol and abs(a[1] - b[1]) <= tol, f"Ranges differ: {a} vs {b}"

def info(msg: str) -> None:
    print("✅", msg)


## Stage 1 — Live vs Home viewport semantics

In [ ]:
# This test assumes the implementation adds/standardizes:
#   - fig.x_range, fig.y_range  (live)
#   - fig.x_range_default, fig.y_range_default  (home/default)

fig = SmartFigure()  # if this fails, your SmartFigure likely needs args; update this cell accordingly.

# Initial: live == home
assert_range_eq(tuple(fig.x_range), tuple(fig.x_range_default))
assert_range_eq(tuple(fig.y_range), tuple(fig.y_range_default))
info("Initial live ranges equal home ranges.")

# Programmatic update: live updates AND home updates
new_x = (0.0, 1.0)
new_y = (-2.0, 2.0)
fig.x_range = new_x
fig.y_range = new_y

assert_range_eq(tuple(fig.x_range), new_x)
assert_range_eq(tuple(fig.y_range), new_y)
assert_range_eq(tuple(fig.x_range_default), new_x)
assert_range_eq(tuple(fig.y_range_default), new_y)
info("Programmatic updates set both live and home ranges.")


## Stage 2 — Plotly relayout updates live ranges only

In [ ]:
# This test assumes the implementation provides a debug hook that simulates plotly relayout:
#   fig._debug_apply_plotly_relayout(delta: dict[str, Any]) -> None
# This avoids needing real UI gestures for deterministic verification.

if not hasattr(fig, "_debug_apply_plotly_relayout"):
    raise AttributeError(
        "SmartFigure must provide _debug_apply_plotly_relayout(delta) for testing. "
        "Implement it as a thin wrapper around the relayout handler."
    )

home_x = tuple(fig.x_range_default)
home_y = tuple(fig.y_range_default)

# Simulate user zoom/pan in Plotly: update live only
fig._debug_apply_plotly_relayout({
    "xaxis.range[0]": 0.25,
    "xaxis.range[1]": 0.75,
})

assert_range_eq(tuple(fig.x_range), (0.25, 0.75))
assert_range_eq(tuple(fig.x_range_default), home_x)  # home unchanged
info("Plotly relayout updates live x_range without changing home.")

fig._debug_apply_plotly_relayout({
    "yaxis.range[0]": -1.0,
    "yaxis.range[1]": 1.0,
})
assert_range_eq(tuple(fig.y_range), (-1.0, 1.0))
assert_range_eq(tuple(fig.y_range_default), home_y)  # home unchanged
info("Plotly relayout updates live y_range without changing home.")


## Stage 3 — Reset axes restores home

In [ ]:
# The Plotly 'reset axes' button often emits autorange=True deltas.
# The implementation must treat this as 'go home' and restore stored defaults.

# First, set a new home via code
fig.x_range = (10.0, 20.0)
fig.y_range = (30.0, 40.0)

# Then simulate a user change away from home
fig._debug_apply_plotly_relayout({"xaxis.range[0]": 12.0, "xaxis.range[1]": 18.0})
fig._debug_apply_plotly_relayout({"yaxis.range[0]": 32.0, "yaxis.range[1]": 38.0})
assert_range_eq(tuple(fig.x_range), (12.0, 18.0))
assert_range_eq(tuple(fig.y_range), (32.0, 38.0))

# Now simulate "reset axes"
fig._debug_apply_plotly_relayout({"xaxis.autorange": True, "yaxis.autorange": True})

assert_range_eq(tuple(fig.x_range), tuple(fig.x_range_default))
assert_range_eq(tuple(fig.y_range), tuple(fig.y_range_default))
assert_range_eq(tuple(fig.x_range), (10.0, 20.0))
assert_range_eq(tuple(fig.y_range), (30.0, 40.0))
info("'Reset axes' restores the stored home/default ranges.")


## Stage 4 — Debounced recompute trigger (flushable)

In [ ]:
# This test assumes the implementation exposes debug stats and a flush hook:
#   fig._debug_viewport_sync_stats() -> dict[str, Any]
#   fig._debug_flush_viewport_debounce() -> None
#
# The recompute should be requested on relayout, but executed after idle.
# For deterministic tests we *do not wait*; we trigger `flush()`.

for attr in ("_debug_viewport_sync_stats", "_debug_flush_viewport_debounce"):
    if not hasattr(fig, attr):
        raise AttributeError(f"Missing required debug hook: {attr}")

stats0 = fig._debug_viewport_sync_stats()
assert isinstance(stats0, dict), "stats must be a dict"

# Generate multiple rapid relayout events
fig._debug_apply_plotly_relayout({"xaxis.range[0]": 11.0, "xaxis.range[1]": 19.0})
fig._debug_apply_plotly_relayout({"xaxis.range[0]": 12.0, "xaxis.range[1]": 18.0})
fig._debug_apply_plotly_relayout({"xaxis.range[0]": 13.0, "xaxis.range[1]": 17.0})

stats1 = fig._debug_viewport_sync_stats()
assert stats1.get("recompute_requests", 0) >= stats0.get("recompute_requests", 0) + 1, stats1

# Must not have necessarily executed yet (depends on delay). We force it.
fig._debug_flush_viewport_debounce()

stats2 = fig._debug_viewport_sync_stats()
assert stats2.get("recompute_runs", 0) >= stats1.get("recompute_runs", 0) + 1, stats2
assert stats2.get("last_recompute_viewport") is not None, stats2

info("Debounced recompute is requestable via relayout and executable via flush().")


## Optional manual sanity check (interactive)

If you're in JupyterLab, you can display the figure below and pan/zoom. This is **not** required for the deterministic tests above, but is useful for human inspection.

In [ ]:
# If SmartFigure has a display method, use it. Otherwise, try to access an underlying plotly widget.
if hasattr(fig, "show"):
    out = fig.show()
    info("Called fig.show(). Interact with the plot if it rendered.")
elif hasattr(fig, "figure"):
    display(fig.figure)  # type: ignore[name-defined]
    info("Displayed fig.figure. Interact with the plot if it rendered.")
else:
    print("No obvious display hook found. If your SmartFigure wraps a FigureWidget, expose it as .figure for convenience.")
